# Laboratorio 2: Armado de un esquema de aprendizaje automático

En el laboratorio final se espera que puedan poner en práctica los conocimientos adquiridos en el curso, trabajando con un conjunto de datos de clasificación.

El objetivo es que se introduzcan en el desarrollo de un esquema para hacer tareas de aprendizaje automático: selección de un modelo, ajuste de hiperparámetros y evaluación.

El conjunto de datos a utilizar está en `./data/loan_data.csv`. Si abren el archivo verán que al principio (las líneas que empiezan con `#`) describen el conjunto de datos y sus atributos (incluyendo el atributo de etiqueta o clase).

Se espera que hagan uso de las herramientas vistas en el curso. Se espera que hagan uso especialmente de las herramientas brindadas por `scikit-learn`.

In [ ]:
import os

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
from sklearn.tree import plot_tree

# AGREGAR TODAS LAS LIBRERIAS QUE FALTAN

# Para dividir el datase en prueba y test
from sklearn.model_selection import train_test_split

# Liberias para calcular las metricas
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score, f1_score
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

# Librerias para ejecutar los metos de prediccion
from sklearn.linear_model import SGDClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn import tree

# Librerias para Analisis y optimizacion de los metors
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

## Carga de datos y división en entrenamiento y evaluación

La celda siguiente se encarga de la carga de datos (haciendo uso de pandas). Estos serán los que se trabajarán en el resto del laboratorio.

In [ ]:
ROOT_PATH = os.path.dirname(os.getcwd())
DATA_PATH = os.path.join(ROOT_PATH, 'data', 'raw')

dataset = pd.read_csv(os.path.join(DATA_PATH, 'loan_data.csv'), comment="#")

dataset.info()

In [ ]:
# Dividimos la informacion del dataset en instancias de X para predecir y armar el modelo
# para eso: analizamos todas las columnas y vemos que el objetivo esta en la primer columna la 0
# por lo tanto tomamos desde la columna 1 todas las restantes :
X = dataset.iloc[:, 1:]
X

In [ ]:
# Una vez armado el dataset de trabajo el nombre de las columnas
columnas = X.columns.values
columnas

In [ ]:
# Dividimos la informacion del dataset en etiquetas resultados
y = dataset.TARGET
y.value_counts()

In [ ]:
# Cantidad de elementos que forman el data frame de datos y el de resultados 
X.shape, y.shape

In [ ]:
# División entre entrenamiento y test usando la funcion de sklearn
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)
X_train.shape, X_test.shape, y_train.shape, y_test.shape 


Documentación:

- https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html

## Ejercicio 1: Descripción de los Datos y la Tarea

Responder las siguientes preguntas:



###1.1 ¿De qué se trata el conjunto de datos?

El conjunto de datos contiene información de los prestamos con garantia hipotecaria otorgados recientemente. Cuenta con información para 5960 préstamos. La variable objetivo (BAD) es una variable binaria que indica si un solicitante finalmente incumplió el crédito o fue gravemente moroso (valor 1). De acuerdo con la información del dataset, solo un 20% pertenece a esta categoría.

###1.2. ¿Cuál es la variable objetivo que hay que predecir? ¿Qué significado tiene?

La variable objetivo es **TARGET** llamada BAD forma parte del dataset por lo tanto debemos extraerla. 
BAD es una variable binaria que indica: 

*   Valor: 1 = El cliente incumple el préstamo.
*   Valor: 0 = El cliente pagó el préstamo.



###1.3. ¿Qué información (atributos) hay disponible para hacer la predicción?

Los atributos que forman parte del dataset y permiten obtener la columna objetivo son:


* **LOAN**    Importe prestamo solicitado
* **MORTDUE** Monto adeudado de hipoteca existente
* **VALUE**   Valor de la propiedad actual	
* **YOJ**     Años en el trabajo actual	
* **DEROG**   Número de informes despectivos importantes	
* **DELINQ**  Número de líneas de crédito morosas
* **CLAGE**   Antigüedad de la línea comercial más antigua en meses
* **NINQ**    Número de líneas de crédito recientes
* **CLNO**    Número de líneas de crédito	
* **DEBTINC** Relación deuda-ingresos	

###1.4. Visualizacion de los atributos que conforman el data set de analisis

In [ ]:
fig = px.violin(dataset, x='TARGET', y='LOAN', color='TARGET', box=True)
fig.update_layout(
    title='Distribución de Deudores con respecto al Importe Prestamo Solicitado',
    xaxis_title='Tipos de Morosos',
    yaxis_title='Importe Prestamo Solicitado')
fig.show()

In [ ]:

fig = px.violin(dataset, x='TARGET', y='MORTDUE', color='TARGET', box=True)
fig.show()
fig = px.violin(dataset, x='TARGET', y='VALUE', color='TARGET', box=True)
fig.show()
fig = px.violin(dataset, x='TARGET', y='YOJ', color='TARGET', box=True)
fig.show()
fig = px.violin(dataset, x='TARGET', y='DEROG', color='TARGET', box=True)
fig.show()
fig = px.violin(dataset, x='TARGET', y='DELINQ', color='TARGET', box=True)
fig.show()
fig = px.violin(dataset, x='TARGET', y='CLAGE', color='TARGET', box=True)
fig.show()
fig = px.violin(dataset, x='TARGET', y='NINQ', color='TARGET', box=True)
fig.show()
fig = px.violin(dataset, x='TARGET', y='CLNO', color='TARGET', box=True)
fig.show()
fig = px.violin(dataset, x='TARGET', y='DEBTINC', color='TARGET', box=True)
fig.show()

In [ ]:
feature = np.array(['LOAN','MORTDUE', 'VALUE', 'YOJ', 'DEROG', 'DELINQ', 'CLAGE', 'NINQ', 'CLNO', 'DEBTINC'])
#colores = ['blue', 'green', 'red', 'cyan', 'magenta', 'yellow', 'black', 'olive', 'orange', 'steelblue']

fig, axs = plt.subplots(5, 2, figsize=(20, 20))
axs = axs.ravel()
for i in range(len(feature)):
    selector = feature[i]
    sns.distplot(dataset[selector] , color="darkblue", ax=axs[i])
    #sns.distplot(dataset[selector] , color=colores[i], ax=axs[i])
    axs[i].set_xlabel(f'Columna {selector}')
fig.tight_layout()
plt.show()

In [ ]:
feature = np.array(['LOAN','MORTDUE', 'VALUE', 'YOJ', 'DEROG', 'DELINQ', 'CLAGE', 'NINQ', 'CLNO', 'DEBTINC'])
#colores = ['blue', 'green', 'red', 'cyan', 'magenta', 'yellow', 'black', 'olive', 'orange', 'steelblue']

fig, axs = plt.subplots(5, 2, figsize=(20, 20))
axs = axs.ravel()
for i in range(len(feature)):
    selector = feature[i]
    sns.boxplot(x = dataset['TARGET'], y =dataset[selector], data=dataset, color="darkblue", ax=axs[i])
    #sns.boxplot(x = dataset['TARGET'], y =dataset[selector], data=dataset, color=colores[i], ax=axs[i])
    axs[i].set_xlabel('Tipos de Morosos')
    axs[i].set_ylabel(f'Columna {selector}')
fig.tight_layout()
plt.show()

In [ ]:
# Realizamos un HEATMAP para verificar la correlacion entre las variable, por eso 
# uso la funcion .corr()
plt.figure (figsize=(16,10))
sns.heatmap( dataset[dataset.columns].corr(), annot=True, fmt='.2g')
plt.title('Correlation entre Variables', fontsize=14)

###1.5. ¿Qué atributos imagina ud. que son los más determinantes para la predicción?

Analizando el mapa de calor de las variables se puede ver que los niveles de correlación entre las variables son bajos, salvo los casos de las variables VALUE con MORTDUE y LOAN. Por lo tanto los atributos disponibles que se estima que pueden ser mas determinantes para la predicción, son:

DEROG - número de informes despectivos importantes

DELINQ - número de líneas de crédito morosas

CLAGE - Antigüedad de la línea comercial más antigua en meses

NINQ - Número de líneas de crédito recientes

DEBTINC - Relación deuda-ingresos

## Ejercicio 2: Predicción con Modelos Lineales

En este ejercicio se entrenarán modelos lineales de clasificación para predecir la variable objetivo.

Para ello, deberán utilizar la clase SGDClassifier de scikit-learn.

Documentación:
- https://scikit-learn.org/stable/modules/sgd.html
- https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.SGDClassifier.html


### Ejercicio 2.1: SGDClassifier con hiperparámetros por defecto

Entrenar y evaluar el clasificador SGDClassifier usando los valores por omisión de scikit-learn para todos los parámetros. Únicamente **fijar la semilla aleatoria** para hacer repetible el experimento.

Evaluar sobre el conjunto de **entrenamiento** y sobre el conjunto de **evaluación**, reportando:
- Accuracy
- Precision
- Recall
- F1
- matriz de confusión

#### 2.1.1 Definicion y Entrenamiento del modelo *ESTANDAR* u *OPTIMIZADO*

In [ ]:
# Utilizamos la función de sklearn para generar el modelo de SGDClasifier en la 
# variable model_class_std con una semilla de 99. 


# SELECCIONAR SI DESEAMOS GENERAR UN MODELO ESTANDAR U OPTIMIZADO

model_class_std = Pipeline([('standardscaler', StandardScaler()), ('model',SGDClassifier(random_state=42))])

#model_class_std = SGDClassifier(random_state=99)

# Realizo el entrenamiento de la variable con la funcion fit y le pasamos los datos de entrenamiento
model_class_std.fit(X_train, y_train)

####2.1.2 Evaluacion sobre conjunto de TRAINING

In [ ]:
# Usamos la funcion predict y le pasamos los dato de train a nuestro modelo para que genere una salida
y_pred_train = model_class_std.predict(X_train)

Calculamos las metricas solicitadas donde comparamos las y_train con las y_pred que son las salidas de nuestro modelo


In [ ]:
# Devuelve con que % de precisión nuestro modelo se acerca al y_test que se tomó del dataset original 
# En este caso el % es alto pero caso contrario tenemos que modificar los parametros de entrenamiento
# de la funcion fit. Si así mismo no llegamos a un buen valor de accurancy tenemos que revisar los 
# datos del modelo que tenemos, verificar los posibles errores, agregar más registros del dataset etc
accuracy_s = accuracy_score(y_train, y_pred_train)
print("El valor de Accuracy Score : ", accuracy_s)

# La lectura de esta matriz debe particularmente ser interprestada como las predicciones en la diagonal izquierda
confusion_m = confusion_matrix(y_train, y_pred_train)
print("La Matriz de Confusión : \n", confusion_m)

In [ ]:
display = ConfusionMatrixDisplay(confusion_matrix=confusion_m)
display.plot() 

In [ ]:
# Esta métrica hace un análisis de la precisión del método propiamente dicho considerando las predicciones
# sobre el total de los datos evaluados en TRAIN. Para esta métrica, considerando los datos de la matriz
# de confusión que se interpreta por sus diagonales. La diagonal son los valores clasificados correctamente, y la antidiagonal son los valores clasificados incorrectamente. 
# todo el dataset analizado, vemos que el modelo para un total 1483 registros:


precision_s = precision_score(y_train, y_pred_train, average= None)
print("El valor de Precision Score : ", precision_s)

Con la métrica de precisión se hace un análisis por columna. Lo que nos dicen los datos es:

*El 86% de los datos etiquetados como cero son ceros correctamente clasificados, mientras que el 14% de los datos etiquetados como cero son unos incorrectamente clasificados como cero.

*El 47% de los datos etiquetados como uno son unos correctamente clasificados, mientras que el 53% de los datos etiquetados como uno son ceros incorrectamente clasificados como unos.  

In [ ]:
# Esta métrica hace un análisis del metodo con respoecto a la categoría y el total
# de datos evaluados en esa categoría. Para esta métrica, en este caso de TRAIN y considerando los 
# datos de la matriz de confusión que se interpreta por sus diagonales, realizamos un analisis horizontal, 
# es decir por CATEGORIA.
 
recall_s = recall_score(y_train, y_pred_train, average= None) 
print("El valor de Recall Score : ", recall_s)

Con la métrica de recall se hace un análisis por fila. Lo que nos dicen los datos es:

*El 93% de los datos con valor cero, han sido correctamente clasificados como cero, mientras que el 7% de los datos con valor cero han sido incorrectamente clasificados como uno.

**El 28% de los datos con valor uno, han sido correctamente clasificados como unos, mientras que el 72% de los datos con valor uno han sido incorrectamente clasificados como ceros.

In [ ]:
# Es un promedio quee se calcula con respecto a las metricas de recall_score y presicion_score
# (recall_s+precision_s)/2
f1_s = f1_score(y_train, y_pred_train, average= None)
print("El valor de F1 Score : ", f1_s)

In [ ]:
# Con este reporte podemos ver todos los parametros que analizamos antes y se arma la matriz de clasificación
print(classification_report(y_train, y_pred_train))

####2.1.3 Evaluacion sobre el conjunto de TEST

In [ ]:
# Usamos la funcion predict y le pasamos los dato de test a nuestro modelo para que genere una salida
y_pred_test = model_class_std.predict(X_test)

Calculamos las métricas solicitadas donde comparamos las y_test con las y_pred que son las salidas de nuestro modelo


In [ ]:
# Devuelve con que % de precisión nuestro modelo se acerca al y_test que se tomó del dataset original 
# En este caso el % es alto pero caso contrario tenemos que modificar los parametros de entrenamiento
# de la funcion fit. Si asi mismo no llegamos a un buen valor de accurancy tenemos que revisar los 
# datos del modelo que tenemos, verificar los posibles errores, agregar mas registros del data set, etc
accuracy_s = accuracy_score(y_test, y_pred_test)
print("El valor de Accuracy Score : ", accuracy_s)

In [ ]:
# La lectura de esta matriz debe particularmente ser interprestada como las predicciones en la diagonal izquierda
# osea el modelo:

confusion_m = confusion_matrix(y_test, y_pred_test)
print("La Matriz de Confusión : \n", confusion_m)

In [ ]:
display = ConfusionMatrixDisplay(confusion_matrix=confusion_m)
display.plot() 

In [ ]:
#Estos datos se interpretan igual que para los datos de train
precision_s = precision_score(y_test, y_pred_test, average= None)
print("El valor de Precision Score : ", precision_s)

In [ ]:
#Estos datos se interpretan igual que para los datos de train
recall_s = recall_score(y_test, y_pred_test, average= None) 
print("El valor de Recall Score : ", recall_s)

In [ ]:
# Es un promedio quee se calcula con respecto a las metricas de recall_score y presicion_score
# (recall_s+precision_s)/2
f1_s = f1_score(y_test, y_pred_test, average= None)
print("El valor de F1 Score : ", f1_s)


In [ ]:
# Con este reporte podemos ver todos los parametros que analizamos antes y se arma la matriz de clasificación
print(classification_report(y_test, y_pred_test))

Vemos que al calcular las métricas para los datos de test, los valores son similares a las métricas obtenidas con los datos de train, por lo cual podemos descartar un problema grave de overfitting en el modelo. 

### Ejercicio 2.2: Ajuste de Hiperparámetros

Seleccionar valores para los hiperparámetros principales del SGDClassifier. Como mínimo, probar diferentes funciones de loss, tasas de entrenamiento y tasas de regularización.

Para ello, usar grid-search y 5-fold cross-validation sobre el conjunto de entrenamiento para explorar muchas combinaciones posibles de valores.

Reportar accuracy promedio y varianza para todas las configuraciones.

Para la mejor configuración encontrada, evaluar sobre el conjunto de **entrenamiento** y sobre el conjunto de **evaluación**, reportando:
- Accuracy
- Precision
- Recall
- F1
- matriz de confusión

Documentación:
- https://scikit-learn.org/stable/modules/grid_search.html
- https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html

#### 2.2.1 Determinar los Hiperparametros del Mejor Modelo +ESTANDAR*

In [ ]:
# Utilizamos la función de sklearn para generar el modelo de análisis con SGDClasifier en la 
# variable model_class simplemente para definir los mejores parametros ya tomo una semilla de 99
model_class = SGDClassifier(random_state=99, eta0=0.0001)

#Observamos los hiperparametros que podemos modificar del modelos para ajustarlo mejor
model_class.get_params()

In [ ]:
# Obtenemos las opciones del hiperparametro de  'loss funtions' posibles para considerar
model_class.loss_functions

In [ ]:
# Armamos una matriz de vectores donde colocamos las alternativas para cada parámetro
# que vamos a considerar. Estos son 'loss', 'learning_rate', 'alpha', 'penalty'
parametros = {
    'random_state':[99, 44, 25],
    'max_iter':[3000, 1200, 2500, 1000 ],
    'loss': ['hinge', 'log', 'squared_loss', 'squared_hinge'],
    'learning_rate':['constant', 'optimal','adaptive'],
    'alpha':[0.01, 0.05, 0.5, 0.001, 0.6, 0.1, 1.0],
    'penalty':['l2', 'l1', 'elasticnet']
    }
parametros

In [ ]:
# Realizamos el análisis del modelo considerando las opciones definidas como posibles 
# configuraciones en el diccionario 'parametros'
cv_class = GridSearchCV(model_class, parametros, scoring ='accuracy', refit=True, cv=5)

# Entrenamos el modelo que calcula los mejores parámetros. Podemos usar la version standard de la 
# definicion de parámetros o los parametros para la versión optimizada
cv_class.fit(X_train, y_train);

In [ ]:
# Realizado el análisis de todas las posibles alternativas de prarámetros con la Clase GridSearchCV
# obtenemos los resultados y los insertamos en un dataframe para su visualización, tomo por orden de mejor a
# peor. Listamos los 6 primeros resultados:
resultados = cv_class.cv_results_
df_resultados = pd.DataFrame(resultados)
df_resultados.sort_values('rank_test_score')[:6]

In [ ]:
# Con estos métodos de la Clase GridSearchCV se obtienen los parámetros para 
# determinar la función que mejor realiza la predicción
cv_class.best_estimator_

In [ ]:
cv_class.best_params_

##### 2.2.1.1 Definicion y Entrenemiento del Mejor Modelo 

In [ ]:
# Mejor modelo SIN la optimizacion
best_model_class = SGDClassifier(alpha=0.1, learning_rate='adaptive', loss='hinge', 
                                 eta0=0.0001, max_iter=3000,  penalty='l1', random_state=99)


In [ ]:
# Realizamos el entrenamiento del modelo con la funcion fit y le pasamos los datos de entrenamiento
best_model_class.fit(X_train, y_train)

##### 2.2.1.2 Evaluacion sobre conjunto de TRAINING

In [ ]:
# Usamos la función predict y le pasamos los dato de test a nuestro modelo para que genere una salida
y_pred_train = best_model_class.predict(X_train)

Calculamos las métricas solicitadas donde comparamos las y_train con las y_pred que son las salidas de nuestro modelo

In [ ]:

accuracy_s = accuracy_score(y_train, y_pred_train)
print("El valor de Accuracy Score : ", accuracy_s)

In [ ]:

confusion_m = confusion_matrix(y_train, y_pred_train)
print("La Matriz de Confusión : \n", confusion_m)

In [ ]:
display = ConfusionMatrixDisplay(confusion_matrix=confusion_m)
display.plot() 

In [ ]:

precision_s = precision_score(y_train, y_pred_train, average= None)
print("El valor de Precision Score : ", precision_s)

In [ ]:

recall_s = recall_score(y_train, y_pred_train, average= None) 
print("El valor de Recall Score : ", recall_s)

In [ ]:
# Es un promedio quee se calcula con respecto a las metricas de recall_score y presicion_score
# (recall_s+precision_s)/2
f1_s = f1_score(y_train, y_pred_train, average= None)
print("El valor de F1 Score : ", f1_s)

In [ ]:
# Con este reporte podemos ver todos los parametros que analiaamos antes y se arma la matriz de clasificación
print(classification_report(y_train, y_pred_train))

##### 2.2.1.3 Evaluacion sobre conjunto de TEST

In [ ]:
# Usamos la funcion predict y le pasamos los datos de test a nuestro modelo para que genere una salida
y_pred_test = best_model_class.predict(X_test)

Calculamos las métricas solicitadas donde comparamos las y_test con las y_pred que son las salidas de nuestro modelo

In [ ]:

accuracy_s = accuracy_score(y_test, y_pred_test)
print("El valor de Accuracy Score : ", accuracy_s)

In [ ]:

confusion_m = confusion_matrix(y_test, y_pred_test)
print("La Matriz de Confusión : \n", confusion_m)

In [ ]:
display = ConfusionMatrixDisplay(confusion_matrix=confusion_m)
display.plot() 

In [ ]:

precision_s = precision_score(y_test, y_pred_test, average= None)
print("El valor de Precision Score : ", precision_s)

In [ ]:

recall_s = recall_score(y_test, y_pred_test, average= None) 
print("El valor de Recall Score : ", recall_s)

In [ ]:
# Es un promedio quee se calcula con respecto a las metricas de recall_score y presicion_score
# (recall_s+precision_s)/2
f1_s = f1_score(y_test, y_pred_test, average= None)
print("El valor de F1 Score : ", f1_s)

In [ ]:
# Con este reporte podemos ver todos los parametros que analizamos antes y se arma la matriz de clasificación
print(classification_report(y_test, y_pred_test))

Como conclusión podemos ver que no hay problemas de overfitting, pero el modelo no es bueno clasificando los valores uno de la variable target (tiene baja precision y bajo recall).

#### 2.2.2 Determinar los Hiperparametros del Mejor Modelo *OPTIMIZADO*

Se tiene que realizar por separado porque el modelo PIPELINE tiene para el modelo 9 de clasificacion SGDClassifier otro formato de prametros

In [ ]:
from scipy import stats

parametros_ppl = {
    'model__loss': ['hinge', 'log', 'squared_loss', 'squared_hinge'],
    'model__learning_rate':['constant', 'optimal','adaptive'],
    'model__alpha': [0.001, 0.0001, 0.00001, 1.0, 10.0],
     }

# Determinados los parámetros realizamos una TRANSFORMACIóN del modelo para optimizar el clasificador
model_class_opt = Pipeline([('standardscaler', StandardScaler()), ('model',SGDClassifier(random_state=42, eta0=0.1))])

# Realizamos el análisis del modelo considerando las opciones definidas como posibles 
# configuración en el diccionario 'parametros'
cv_class_opt = GridSearchCV(model_class_opt, parametros_ppl, scoring ='accuracy', refit=True, cv=5)

# Entrenamos el modelo que  calcula los mejores parámetros. Podemos usar la version standard de la 
# definición de parametros o los parámetros para la versión optimizada
cv_class_opt.fit(X_train, y_train);

In [ ]:
# Realizado el análisis de todas las posibles alternativas de parámetros con la Clase GridSearchCV
# obtenemos los resultados y los insertamos en un data frame para su visualización, tomando por orden de mejor a
# peor los 6 primeros resultados
resultados = cv_class_opt.cv_results_
df_resultados = pd.DataFrame(resultados)
df_resultados.sort_values('rank_test_score')[:6]

In [ ]:

cv_class_opt.best_estimator_

In [ ]:
cv_class_opt.best_params_

##### 2.2.2.1 Definicion y Entrenemiento del Mejor Modelo


In [ ]:
# Mejor modelo CON la optimización 
best_model_class_opt = Pipeline([
    ('standardscaler', StandardScaler()), 
    ('model',SGDClassifier(
            alpha=0.001, loss='hinge', max_iter=3000, 
            random_state=42, eta0=0.1, learning_rate='adaptive', penalty='l2'
        )
    )
])

In [ ]:
# Realizamos el entrenamiento del modelo con la funcion fit y le pasamos los datos de entrenamiento
best_model_class_opt.fit(X_train, y_train)

##### 2.2.2.2 Evaluacion sobre conjunto de TRAINING

In [ ]:
# Usamos la funcion predict y le pasamos los dato de test a nuestro modelo para que genere una salida
y_pred_train = best_model_class_opt.predict(X_train)



```
# This is formatted as code
```

Calculamos las métricas solicitadas donde comparamos las y_train con las y_pred que son las salidas de nuestro modelo

In [ ]:

accuracy_s = accuracy_score(y_train, y_pred_train)
print("El valor de Accuracy Score : ", accuracy_s)

In [ ]:

confusion_m = confusion_matrix(y_train, y_pred_train)
print("La Matriz de Confusión : \n", confusion_m)

In [ ]:
display = ConfusionMatrixDisplay(confusion_matrix=confusion_m)
display.plot() 

In [ ]:

precision_s = precision_score(y_train, y_pred_train, average= None)
print("El valor de Precision Score : ", precision_s)

In [ ]:

recall_s = recall_score(y_train, y_pred_train, average= None) 
print("El valor de Recall Score : ", recall_s)

In [ ]:
# Es un promedio quee se calcula con respecto a las metricas de recall_score y presicion_score
# (recall_s+precision_s)/2
f1_s = f1_score(y_train, y_pred_train, average= None)
print("El valor de F1 Score : ", f1_s)

In [ ]:
#Matriz de clasificación
print(classification_report(y_train, y_pred_train))

Con estos parámetros y modelo optimizado, vemos que mejora considerablemente la métrica de precision para la categoría de unos, no así el recall.



##### 2.2.2.3 Evaluacion sobre conjunto de TEST

In [ ]:
# Usamos la funcion predict y le pasamos los dato de test a nuestro modelo para que genere una salida
y_pred_test = best_model_class_opt.predict(X_test)

Calculamos las métricas solicitadas donde comparamos las y_test con las y_pred que son las salidas de nuestro modelo

In [ ]:

accuracy_s = accuracy_score(y_test, y_pred_test)
print("El valor de Accuracy Score : ", accuracy_s)

In [ ]:

confusion_m = confusion_matrix(y_test, y_pred_test)
print("La Matriz de Confusión : \n", confusion_m)

In [ ]:
display = ConfusionMatrixDisplay(confusion_matrix=confusion_m)
display.plot() 

In [ ]:

precision_s = precision_score(y_test, y_pred_test, average= None)
print("El valor de Precision Score : ", precision_s)

In [ ]:

recall_s = recall_score(y_test, y_pred_test, average= None) 
print("El valor de Recall Score : ", recall_s)

In [ ]:
# Es un promedio quee se calcula con respecto a las metricas de recall_score y presicion_score
# (recall_s+precision_s)/2
f1_s = f1_score(y_test, y_pred_test, average= None)
print("El valor de F1 Score : ", f1_s)

In [ ]:
# Matriz de clasificación
print(classification_report(y_test, y_pred_test))

También con los datos de test se observa la mejora en el precision de la categoría uno, aunque no así en recall. También vemos que la métrica precision empeora en la categoría cero.

## Ejercicio 3: Árboles de Decisión

En este ejercicio se entrenarán árboles de decisión para predecir la variable objetivo.

Para ello, deberán utilizar la clase DecisionTreeClassifier de scikit-learn.

Documentación:
- https://scikit-learn.org/stable/modules/tree.html
  - https://scikit-learn.org/stable/modules/tree.html#tips-on-practical-use
- https://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeClassifier.html
- https://scikit-learn.org/stable/auto_examples/tree/plot_unveil_tree_structure.html

### Ejercicio 3.1: DecisionTreeClassifier con hiperparámetros por defecto

Entrenar y evaluar el clasificador DecisionTreeClassifier usando los valores por omisión de scikit-learn para todos los parámetros. Únicamente **fijar la semilla aleatoria** para hacer repetible el experimento.

Evaluar sobre el conjunto de **entrenamiento** y sobre el conjunto de **evaluación**, reportando:
- Accuracy
- Precision
- Recall
- F1
- matriz de confusión


#### 3.1.1 Definicion y Entrenamiento del modelo

In [ ]:
# Utilizamos la función de sklearn para generar el modelo de DecisionTreeClassifier en la 
#variable model con una semilla de 99

model_tree_std = DecisionTreeClassifier(random_state=99)

# Realizamos el entrenamiento de la variable con la funcion fit y le pasamos los datos de entrenamiento
model = model_tree_std.fit(X_train,y_train)

####3.1.2 Evaluacion sobre conjunto de TRAINING

In [ ]:
# Usamos la funcion predict y le pasamos los datos de test a nuestro modelo para que genere una salida
y_pred_train = model_tree_std.predict(X_train)

Calculamos las metricas solicitadas donde comparamos las y_train con las y_pred que son las salidas de nuestro modelo


In [ ]:
#Calculamos las métricas del modelo
accuracy_s = accuracy_score(y_train, y_pred_train)
print("El valor de Accuracy Score : ", accuracy_s)

In [ ]:
# La lectura de esta matriz debe particularmente ser interprestada como las predicciones en la diagonal (correctas)y en la antidiagonal (incorrectas)
confusion_m = confusion_matrix(y_train, y_pred_train)
print("La Matriz de Confusión : \n", confusion_m)

In [ ]:
display = ConfusionMatrixDisplay(confusion_matrix=confusion_m)
display.plot() 

In [ ]:
precision_s = precision_score(y_train, y_pred_train, average= None)
print("El valor de Precision Score : ", precision_s)

In [ ]:

recall_s = recall_score(y_train, y_pred_train, average= None) 
print("El valor de Recall Score : ", recall_s)

In [ ]:
# Es un promedio quee se calcula con respecto a las metricas de recall_score y presicion_score
# (recall_s+precision_s)/2
f1_s = f1_score(y_train, y_pred_train, average= None)
print("El valor de F1 Score : ", f1_s)

In [ ]:
# Con este reporte podemos ver todos los parametros que analisamos antes y se arma la matriz de clasificacion
print(classification_report(y_train, y_pred_train))

Lo llamativo es que las métrcas dan un 100% de accuracy, precision y recall (para ambos valores). Podría ser un problema de overfitting. A continuación analizamos las métricas con los datos de test. 



####3.1.3 Evaluación sobre el conjunto de TEST

In [ ]:
# Usamos la función predict y le pasamos los dato de test a nuestro modelo para que genere una salida
y_pred_test = model_tree_std.predict(X_test)

Calculamos las metricas solicitadas donde comparamos las y_test con las y_pred que son las salidas de nuestro modelo

In [ ]:

accuracy_s = accuracy_score(y_test, y_pred_test)
print("El valor de Accuracy Score : ", accuracy_s)

In [ ]:

confusion_m = confusion_matrix(y_test, y_pred_test)
print("La Matriz de Confusión : \n", confusion_m)

In [ ]:
display = ConfusionMatrixDisplay(confusion_matrix=confusion_m)
display.plot() 

In [ ]:

precision_s = precision_score(y_test, y_pred_test, average= None)
print("El valor de Precision Score : ", precision_s)

In [ ]:

recall_s = recall_score(y_test, y_pred_test, average= None) 
print("El valor de Recall Score : ", recall_s)

In [ ]:
# Es un promedio quee se calcula con respecto a las metricas de recall_score y presicion_score
# (recall_s+precision_s)/2
f1_s = f1_score(y_test, y_pred_test, average= None)
print("El valor de F1 Score : ", f1_s)

In [ ]:
# Con este reporte podemos ver todos los parametros que analizamos antes y se arma la matriz de clasificación
print(classification_report(y_test, y_pred_test))

Para el caso de los datos de test el modelo no es tan bueno, sobretodo para clasificar los valores unos, por lo cual podemos pensar que con los hiperparámetros por default del DecisionTreeClassifier estamos generando un problema de overfitting. 

####3.1.4 Diseño del Arbol de Desicion del modelo

In [ ]:
plt.figure(figsize=[35, 35])
plot_tree(model_tree_std, filled=True, fontsize=10);

### Ejercicio 3.2: Ajuste de Hiperparámetros

Seleccionar valores para los hiperparámetros principales del DecisionTreeClassifier. Como mínimo, probar diferentes criterios de partición (criterion), profundidad máxima del árbol (max_depth), y cantidad mínima de samples por hoja (min_samples_leaf).

Para ello, usar grid-search y 5-fold cross-validation sobre el conjunto de entrenamiento para explorar muchas combinaciones posibles de valores.

Reportar accuracy promedio y varianza para todas las configuraciones.

Para la mejor configuración encontrada, evaluar sobre el conjunto de **entrenamiento** y sobre el conjunto de **evaluación**, reportando:
- Accuracy
- Precision
- Recall
- F1
- matriz de confusión


Documentación:
- https://scikit-learn.org/stable/modules/grid_search.html
- https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html

#### 3.2.1 Determinar los Hiperparametros del Mejor Modelo

In [ ]:
# Utilizamos la función de sklearn para generar el modelo de DecisionTreeClassifier en la 
#variable model_tree para definir los mejores parametros ya tomo una semilla de 99
model_tree = DecisionTreeClassifier(random_state=99)

#Observamos los hiperparametros que podemos modificar del modelo para ajustarlo mejor
model_tree.get_params()

In [ ]:
# Armamos una matriz de vectores donde colocamos las alternativas para cada parámetro
# que vamos a considerar. Son 'criterion', 'max_depth', 'min_samples_leaf', 'random_state'
parametros = {
    'random_state':[99, 56, 23, 45],
    'criterion': ['entropy', 'gini'], 
    'max_depth':[5, 6, 7, 8, 9, 10, 20, 50, 100, 200],
    'min_samples_leaf': [1, 5, 10, 50]}
parametros    

In [ ]:
cv_tree = GridSearchCV(model_tree, parametros, scoring ='accuracy', refit=True, cv=5)
cv_tree.fit(X_train, y_train)

In [ ]:
# Realizado el analisis de todas las posibles alternativas de prarametros en la Clase GridSearchCV
# tomamos los resultados y los insertamos en un dataframe para su visualización
resultados = cv_tree.cv_results_
df_resultados = pd.DataFrame(resultados)
df_resultados.sort_values('rank_test_score')[:6]

In [ ]:
# Con este parametro de la Clase GridSearchCV podemos tomar los parametros que calculo para 
# determinar los hiperparámetros que mejor predicen los datos
cv_tree.best_estimator_

In [ ]:
cv_tree.best_params_

#### 3.2.2 Definicion y Entrenemiento del Mejor Modelo

In [ ]:
best_model_tree = DecisionTreeClassifier(criterion='gini', max_depth=7, min_samples_leaf=10, random_state=56)

In [ ]:
# Realizamos el entrenamiento de la variable con la funcion fit y le pasamos los datos de entrenamiento
best_model_tree.fit(X_train, y_train)

#### 3.2.2 Evaluacion sobre conjunto de TRAINING

In [ ]:
# Usamos la funcion predict y le pasamos los datos de entrenamiento a nuestro modelo para que genere una salida
y_pred_train = best_model_tree.predict(X_train)

Calculamos las metricas solicitadas donde comparamos las y_train con las y_pred que son las salidas de nuestro modelo

In [ ]:
#Calculamos las métricas
accuracy_s = accuracy_score(y_train, y_pred_train)
print("El valor de Accuracy Score : ", accuracy_s)

In [ ]:

confusion_m = confusion_matrix(y_train, y_pred_train)
print("La Matriz de Confusión : \n", confusion_m)

In [ ]:
display = ConfusionMatrixDisplay(confusion_matrix=confusion_m)
display.plot() 

In [ ]:

precision_s = precision_score(y_train, y_pred_train, average= None)
print("El valor de Precision Score : ", precision_s)

In [ ]:

recall_s = recall_score(y_train, y_pred_train, average= None) 
print("El valor de Recall Score : ", recall_s)

In [ ]:
# Es un promedio quee se calcula con respecto a las metricas de recall_score y presicion_score
# (recall_s+precision_s)/2
f1_s = f1_score(y_train, y_pred_train, average= None)
print("El valor de F1 Score : ", f1_s)

In [ ]:
# Con este reporte podemos ver todos los parámetros que analizamos antes y se arma la matriz de clasificación
print(classification_report(y_train, y_pred_train))

Vemos que el modelo clasifica mejor los ceros que los unos. No parece haber problemas de overfitting en este caso. A continuación vamos a analizar los resultados sobre el conjunto de test para confirmarlo.

#### 3.2.3 Evaluacion sobre conjunto de TEST

In [ ]:
# Usamos la función predict y le pasamos los dato de test a nuestro modelo para que genere una salida
y_pred_test = best_model_tree.predict(X_test)

Calculamos las metricas solicitadas donde comparamos las y_test con las y_pred que son las salidas de nuestro modelo

In [ ]:

accuracy_s = accuracy_score(y_test, y_pred_test)
print("El valor de Accuracy Score : ", accuracy_s)

In [ ]:

confusion_m = confusion_matrix(y_test, y_pred_test)
print("La Matriz de Confusión : \n", confusion_m)

In [ ]:
display = ConfusionMatrixDisplay(confusion_matrix=confusion_m)
display.plot() 

In [ ]:

precision_s = precision_score(y_test, y_pred_test, average= None)
print("El valor de Precision Score : ", precision_s)

In [ ]:

recall_s = recall_score(y_test, y_pred_test, average= None) 
print("El valor de Recall Score : ", recall_s)

In [ ]:
# Es un promedio quee se calcula con respecto a las metricas de recall_score y presicion_score
# (recall_s+precision_s)/2
f1_s = f1_score(y_test, y_pred_test, average= None)
print("El valor de F1 Score : ", f1_s)

In [ ]:
# Con este reporte podemos ver todos los parametros que analizamos antes y se arma la matriz de clasificación
print(classification_report(y_test, y_pred_test))

Vemos que los valores de las métricas son similares (aunque ligeramente inferiores) a los obtenidos con los datos de train.

####3.2.4 Diseño del Arbol de Desicion del modelo

In [ ]:
plt.figure(figsize=[35, 35])
plot_tree(best_model_tree, filled=True, fontsize=10);